In [6]:
import requests
import json
import numpy as np
import pandas as pd
from datetime import datetime 
from datetime import timedelta

In [9]:
def get_stock_rank_info(his_day, tickers, step_back_days):
    today_time=datetime.now() 
    ini_time=datetime(1970,1,1,0,0,0,0)
    start_time = today_time - timedelta(days=his_day)
    start_sec = (start_time-ini_time).days*3600*24
    end_sec = (today_time-ini_time).days*3600*24
    
    ticker_price = pd.DataFrame()
    #get data and calculate the rank based on stock returns 
    for t in tickers:
        print("start to calculate return and rank for ticker " + t)
        site = f'https://query1.finance.yahoo.com/v7/finance/download/{t}?period1={start_sec}&period2={end_sec}&interval=1d&events=history&includeAdjustedClose=true'
        #print(site)
        #ticker_tmp = pd.read_csv(site,parse_dates=["Date"]).set_index("Date")
        ticker_tmp = pd.read_csv(site,parse_dates=["Date"])
        ticker_tmp["Ticker"] = t
        ticker_tmp = ticker_tmp.set_index(["Date","Ticker"])
        tmp_df = ticker_tmp["Close"].to_frame()
        for i in step_back_days:
            tmp_df_step1 = tmp_df.diff(periods = i)
            a = "_" + str(i)+"_daysDiff"
            ticker_tmp = ticker_tmp.join(tmp_df_step1, on=None, how='left', lsuffix='', rsuffix=a, sort=True)
            b = "Return_" + str(i)+"_days"
            c = "Close" + a
            ticker_tmp[b] = ticker_tmp[c]/(ticker_tmp["Close"]-ticker_tmp[c]) 
        #ticker_tmp["Ticker"] = t
        ticker_price = pd.concat([ticker_price,ticker_tmp])
        
    for i in step_back_days:    
        ticker_price[str(i)+"_days_rank"] = ticker_price.groupby('Date')["Return_" + str(i)+"_days"].rank(method='first', ascending=False)
    del tmp_df, tmp_df_step1
    
    ticker_price2 = ticker_price.copy()
    #calculate the average rank for stop with time step s
    for s in step_back_days:
        t_part_total = pd.DataFrame()
        for t in tickers:
            print("start to calculate average rank for stock " + t+" with time pace " + str(s) + " days")
            idx = pd.IndexSlice
            ticker_tmp = ticker_price2.loc[idx[:, t], :]
            ticker_tmp = ticker_tmp.sort_index(level=0,ascending=False)
            t_part = pd.DataFrame(columns= ["Date","Ticker",str(s)+"_day_rank_mean"])
            for i in range(0,len(ticker_tmp[str(s)+"_days_rank"].to_frame().dropna())-s-1):
                ticker_tmp2 = ticker_tmp.iloc[i:i+s]
                ticker_mean = ticker_tmp2[str(s)+"_days_rank"].mean()
                max_day = ticker_tmp2.index[0][0]
                t_name = t
                t_part.loc[i] = [max_day,t_name,ticker_mean]
            t_part_total = pd.concat([t_part_total,t_part])
            
        t_part_total = t_part_total.reset_index(drop=True)
        t_part_total = t_part_total.set_index(["Date","Ticker"])
        ticker_price2 = ticker_price2.join(t_part_total, on=None, how='left', lsuffix='', rsuffix='', sort=False)
    ticker_price2 = ticker_price2.sort_index(level=[1,0],ascending=False)
   #calculating the 7 days average rank for each time step 
    for s in step_back_days:
        t_part_total = pd.DataFrame()
        for t in tickers:
            print("start to calculate average 7 days rank for stock " + t+" with time pace " + str(s) + " days")
            idx = pd.IndexSlice
            ticker_tmp = ticker_price2.loc[idx[:, t], :]
            ticker_tmp = ticker_tmp.sort_index(level=0,ascending=False)
            t_part = pd.DataFrame(columns= ["Date","Ticker",str(s)+"days_forward_return_"+"_7_days_rank_mean"])
            for i in range(0,len(ticker_tmp[str(s)+"_days_rank"].to_frame().dropna())-7-1):
                ticker_tmp2 = ticker_tmp.iloc[i:i+7]
                ticker_mean = ticker_tmp2[str(s)+"_days_rank"].mean()
                max_day = ticker_tmp2.index[0][0]
                t_name = t
                t_part.loc[i] = [max_day,t_name,ticker_mean]
            t_part_total = pd.concat([t_part_total,t_part])
            
        t_part_total = t_part_total.reset_index(drop=True)
        t_part_total = t_part_total.set_index(["Date","Ticker"])
        ticker_price2 = ticker_price2.join(t_part_total, on=None, how='left', lsuffix='', rsuffix='', sort=False)  
    ticker_price2 = ticker_price2.sort_index(level=[1,0],ascending=False)
    return ticker_price2

def get_day_shifts(df_total, tickers, shift_days):
    idx = pd.IndexSlice
    t_total = pd.DataFrame()
    for t in tickers:
        ticker_tmp = df_total.loc[idx[:, t], :]
        ticker_tmp = ticker_tmp.sort_index(level=0,ascending=False)
        ticker_tmp = ticker_tmp.shift(periods = -1*shift_days)
        t_total = pd.concat([t_total, ticker_tmp], axis=0)
    df_total = df_total.join(t_total, on=None, how='left', lsuffix='', rsuffix='_'+str(shift_days)+'_'+'days_shift_', sort=False)
    df_total = df_total.sort_index(level=[1,0],ascending=False)
    return df_total

In [3]:
his_day = 1200 #define how many days we get tickers history
tickers = ["AXP","AMGN","AAPL","BA","CAT","CSCO","CVX","GS","HD","HON","IBM","INTC","JNJ","KO","JPM","MCD","MMM","MRK","MSFT","NKE","PG","TRV","UNH","CRM","VZ","WBA","WMT","DIS","DOW"] #define tickers interested
step_back_days = [5,23,45,66,133,265] #define time pace we want to do the return and rank
df_rank = get_stock_rank_info(his_day, tickers, step_back_days)
df_rank.to_csv("output_temp.csv")


    

start to calculate return and rank for ticker AXP
start to calculate return and rank for ticker AMGN
start to calculate return and rank for ticker AAPL
start to calculate return and rank for ticker BA
start to calculate return and rank for ticker CAT
start to calculate return and rank for ticker CSCO
start to calculate return and rank for ticker CVX
start to calculate return and rank for ticker GS
start to calculate return and rank for ticker HD
start to calculate return and rank for ticker HON
start to calculate return and rank for ticker IBM
start to calculate return and rank for ticker INTC
start to calculate return and rank for ticker JNJ
start to calculate return and rank for ticker KO
start to calculate return and rank for ticker JPM
start to calculate return and rank for ticker MCD
start to calculate return and rank for ticker MMM
start to calculate return and rank for ticker MRK
start to calculate return and rank for ticker MSFT
start to calculate return and rank for ticker NKE

start to calculate average rank for stock JNJ with time pace 66 days
start to calculate average rank for stock KO with time pace 66 days
start to calculate average rank for stock JPM with time pace 66 days
start to calculate average rank for stock MCD with time pace 66 days
start to calculate average rank for stock MMM with time pace 66 days
start to calculate average rank for stock MRK with time pace 66 days
start to calculate average rank for stock MSFT with time pace 66 days
start to calculate average rank for stock NKE with time pace 66 days
start to calculate average rank for stock PG with time pace 66 days
start to calculate average rank for stock TRV with time pace 66 days
start to calculate average rank for stock UNH with time pace 66 days
start to calculate average rank for stock CRM with time pace 66 days
start to calculate average rank for stock VZ with time pace 66 days
start to calculate average rank for stock WBA with time pace 66 days
start to calculate average rank for 

start to calculate average 7 days rank for stock INTC with time pace 23 days
start to calculate average 7 days rank for stock JNJ with time pace 23 days
start to calculate average 7 days rank for stock KO with time pace 23 days
start to calculate average 7 days rank for stock JPM with time pace 23 days
start to calculate average 7 days rank for stock MCD with time pace 23 days
start to calculate average 7 days rank for stock MMM with time pace 23 days
start to calculate average 7 days rank for stock MRK with time pace 23 days
start to calculate average 7 days rank for stock MSFT with time pace 23 days
start to calculate average 7 days rank for stock NKE with time pace 23 days
start to calculate average 7 days rank for stock PG with time pace 23 days
start to calculate average 7 days rank for stock TRV with time pace 23 days
start to calculate average 7 days rank for stock UNH with time pace 23 days
start to calculate average 7 days rank for stock CRM with time pace 23 days
start to cal

start to calculate average 7 days rank for stock BA with time pace 265 days
start to calculate average 7 days rank for stock CAT with time pace 265 days
start to calculate average 7 days rank for stock CSCO with time pace 265 days
start to calculate average 7 days rank for stock CVX with time pace 265 days
start to calculate average 7 days rank for stock GS with time pace 265 days
start to calculate average 7 days rank for stock HD with time pace 265 days
start to calculate average 7 days rank for stock HON with time pace 265 days
start to calculate average 7 days rank for stock IBM with time pace 265 days
start to calculate average 7 days rank for stock INTC with time pace 265 days
start to calculate average 7 days rank for stock JNJ with time pace 265 days
start to calculate average 7 days rank for stock KO with time pace 265 days
start to calculate average 7 days rank for stock JPM with time pace 265 days
start to calculate average 7 days rank for stock MCD with time pace 265 days
s

In [10]:
df_rank2 = df_rank.iloc[:,-6:]
df_rank2 = get_day_shifts(df_rank2, tickers, 22)

df_rank3 = df_rank.iloc[:,-6:]
df_rank3 = get_day_shifts(df_rank3, tickers, 27)
df_rank3 = df_rank3.iloc[:,-6:]

df_rank4 = df_rank.iloc[:,-6:]
df_rank4 = get_day_shifts(df_rank4, tickers, 32)
df_rank4 = df_rank4.iloc[:,-6:]

df_rank5 = df_rank.iloc[:,-6:]
df_rank5 = get_day_shifts(df_rank5, tickers, 37)
df_rank5 = df_rank5.iloc[:,-6:]

df_rank6 = df_rank.iloc[:,-6:]
df_rank6 = get_day_shifts(df_rank6, tickers, 42)
df_rank6 = df_rank6.iloc[:,-6:]

df_rank7 = df_rank.iloc[:,-6:]
df_rank7 = get_day_shifts(df_rank7, tickers, 87)
df_rank7 = df_rank7.iloc[:,-6:]

df_rank8 = df_rank.iloc[:,-6:]
df_rank8 = get_day_shifts(df_rank8, tickers, 155)
df_rank8 = df_rank8.iloc[:,-6:]

df_rank9 = df_rank.iloc[:,-6:]
df_rank9 = get_day_shifts(df_rank9, tickers, 287)
df_rank9 = df_rank9.iloc[:,-6:]

df_rank2 = df_rank2.join(df_rank3, on=None, how='left', lsuffix='', rsuffix='', sort=False)
df_rank2 = df_rank2.join(df_rank4, on=None, how='left', lsuffix='', rsuffix='', sort=False)
df_rank2 = df_rank2.join(df_rank5, on=None, how='left', lsuffix='', rsuffix='', sort=False)
df_rank2 = df_rank2.join(df_rank6, on=None, how='left', lsuffix='', rsuffix='', sort=False)
df_rank2 = df_rank2.join(df_rank7, on=None, how='left', lsuffix='', rsuffix='', sort=False)
df_rank2 = df_rank2.join(df_rank8, on=None, how='left', lsuffix='', rsuffix='', sort=False)
df_rank2 = df_rank2.join(df_rank9, on=None, how='left', lsuffix='', rsuffix='', sort=False)

df_rank2 = df_rank2.dropna()
del df_rank3,df_rank4,df_rank5,df_rank6,df_rank7,df_rank8,df_rank9

#data manipulation part ends here

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.metrics import accuracy_score
import category_encoders as ce
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error

In [12]:
#starting the SVM part

#this part we are using random split to split the data
df_rank3 = df_rank2.reset_index()

#encoding categorical variable
encoder = ce.BackwardDifferenceEncoder(cols=['Ticker'])
df_rank3 = encoder.fit_transform(df_rank3)

#set X and Y 
X = df_rank3.drop(['Date','5days_forward_return__7_days_rank_mean','23days_forward_return__7_days_rank_mean'], axis=1)

y = df_rank3['23days_forward_return__7_days_rank_mean']

#data spliting
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
X_train.shape, X_test.shape

#scaling data
scaler = MinMaxScaler()
X_train_2 = scaler.fit_transform(X_train)
X_test_2 = scaler.transform(X_test)

In [13]:
#running the regression
svr_rbf = SVR(kernel='rbf', C=50, gamma=0.1)
svr_lin = SVR(kernel='linear', C=50)
svr_poly = SVR(kernel='poly', C=50, degree=3)

print("start rbf fit")
y_rbf = svr_rbf.fit(X_train_2, y_train)
print("start lin fit")
y_lin = svr_lin.fit(X_train_2, y_train)
print("start poly fit")
y_poly = svr_poly.fit(X_train_2, y_train)

print("start rbf pred test")
y_rbf_pred_test = y_rbf.predict(X_test_2)
print("start lin pred test")
y_lin_pred_test = y_lin.predict(X_test_2)
print("start poly pred test")
y_poly_pred_test = y_poly.predict(X_test_2)

print("start rbf pred train")
y_rbf_pred_train = y_rbf.predict(X_train_2)
print("start lin pred train")
y_lin_pred_train = y_lin.predict(X_train_2)
print("start poly pred train")
y_poly_pred_train = y_poly.predict(X_train_2)




start rbf fit
start lin fit
start poly fit


In [19]:

print('MAPE for training data rbf: ', mean_absolute_error(y_train,y_rbf_pred_train)*100, '%')
print('MAPE for training data lin: ', mean_absolute_error(y_train,y_lin_pred_train)*100, '%')
print('MAPE for training data poly: ', mean_absolute_error(y_train,y_poly_pred_train)*100, '%')

print('MAPE for testing data rbf: ', mean_absolute_error(y_test,y_rbf_pred_test)*100, '%')
print('MAPE for testing data lin: ', mean_absolute_error(y_test,y_lin_pred_test)*100, '%')
print('MAPE for testing data poly: ', mean_absolute_error(y_test,y_poly_pred_test)*100, '%')

MAPE for training data rbf:  25.364022987239498 %
MAPE for training data lin:  186.18790478960088 %
MAPE for training data poly:  25.388739516840392 %
MAPE for testing data rbf:  40.73687511016347 %
MAPE for testing data lin:  195.12602383114657 %
MAPE for testing data poly:  43.18808895964207 %


In [70]:
#starting the SVM part

#this part we are using date to split the data

#this part we are using random split to split the data
df_rank3 = df_rank2.reset_index()

#encoding categorical variable
encoder = ce.BackwardDifferenceEncoder(cols=['Ticker'])
df_rank3 = encoder.fit_transform(df_rank3)

df_train = df_rank3.loc[df_rank3['Date'] < '2022-02-01']
df_test = df_rank3.loc[df_rank3['Date'] >= '2022-02-01']


#set X and Y 
X_train = df_train.drop(['Date','5days_forward_return__7_days_rank_mean','23days_forward_return__7_days_rank_mean'], axis=1)
y_train = df_train['23days_forward_return__7_days_rank_mean']

X_test = df_test.drop(['Date','5days_forward_return__7_days_rank_mean','23days_forward_return__7_days_rank_mean'], axis=1)
y_test = df_test['23days_forward_return__7_days_rank_mean']

print(X_train.shape, X_test.shape)

#scaling data
scaler = MinMaxScaler()
X_train_2 = scaler.fit_transform(X_train)
X_test_2 = scaler.transform(X_test)



(5092, 81) (2668, 81)


In [73]:
#running the regression
svr_rbf = SVR(kernel='rbf', C=10, gamma=0.1)
svr_lin = SVR(kernel='linear', C=10)
svr_poly = SVR(kernel='poly', C=10, degree=3)

print("start rbf fit")
y_rbf = svr_rbf.fit(X_train_2, y_train)
print("start lin fit")
y_lin = svr_lin.fit(X_train_2, y_train)
print("start poly fit")
y_poly = svr_poly.fit(X_train_2, y_train)

print("start rbf pred test")
y_rbf_pred_test = y_rbf.predict(X_test_2)
print("start lin pred test")
y_lin_pred_test = y_lin.predict(X_test_2)
print("start poly pred test")
y_poly_pred_test = y_poly.predict(X_test_2)

print("start rbf pred train")
y_rbf_pred_train = y_rbf.predict(X_train_2)
print("start lin pred train")
y_lin_pred_train = y_lin.predict(X_train_2)
print("start poly pred train")
y_poly_pred_train = y_poly.predict(X_train_2)



print('MAPE for training data rbf: ', mean_absolute_error(y_train,y_rbf_pred_train)*100, '%')
print('MAPE for training data lin: ', mean_absolute_error(y_train,y_lin_pred_train)*100, '%')
print('MAPE for training data poly: ', mean_absolute_error(y_train,y_poly_pred_train)*100, '%')

print('MAPE for testing data rbf: ', mean_absolute_error(y_test,y_rbf_pred_test)*100, '%')
print('MAPE for testing data lin: ', mean_absolute_error(y_test,y_lin_pred_test)*100, '%')
print('MAPE for testing data poly: ', mean_absolute_error(y_test,y_poly_pred_test)*100, '%')

start rbf fit
start lin fit
start poly fit
start rbf pred test
start lin pred test
start poly pred test
start rbf pred train
start lin pred train
start poly pred train
MAPE for training data rbf:  44.49402508104765 %
MAPE for training data lin:  190.29408652778145 %
MAPE for training data poly:  38.20476827266505 %
MAPE for testing data rbf:  292.05833922882704 %
MAPE for testing data lin:  228.19565699093206 %
MAPE for testing data poly:  360.7926416798447 %


In [75]:
df_rank2.to_csv("output_3.csv")